#Setup

In [ ]:
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import json
import pandas as pd
from datetime import datetime
import pdb 
from google.colab import drive
import pandas as pd
from psutil import virtual_memory
import os
from zipfile import ZipFile

In [ ]:
class SetUpColab():

  def __init__(self):
    pass
  
  #Determines how much ram the runtime has
  @staticmethod
  def ram_runtime():
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
      print('Not using a high-RAM runtime')
    else:
      print('You are using a high-RAM runtime!')
  
  @staticmethod
  def mount_google_drive():
    drive.mount('/content/gdrive')
  
  #Sets up environement for use with kaggle api
  @staticmethod
  def set_up_kaggle():
    !pip uninstall -y kaggle
    !pip install --upgrade pip
    !pip install kaggle==1.5.6
    !mkdir .kaggle

    token = {"username":"nicholasmagal","key":"9bf671834d75b58fac2b037da15f4cf0"}
    with open('/content/.kaggle/kaggle.json', 'w') as file:
      json.dump(token, file)
    
    for i in range(2):
      !chmod 600 /content/.kaggle/kaggle.json
      !cp /content/.kaggle/kaggle.json /root/.kaggle/
      !kaggle config set -n path -v /content
  
  @staticmethod
  def change_dir(path):
    os.chdir(path)
  
  #Calls above methods to do a complete Collab setup, ready to run ml models :D Note may want to change this per competition
  @staticmethod
  def complete_set_up():
    SetUpColab.ram_runtime()
    #SetUpColab.mount_google_drive()
    SetUpColab.set_up_kaggle()

In [ ]:
SetUpColab.complete_set_up()
data_url = 'idl-fall2021-hw1p2'
data_path = '/content/competitions/' + data_url
!kaggle competitions download -c idl-fall2021-hw1p2
SetUpColab.change_dir(data_path)
!unzip idl-fall2021-hw1p2.zip

#Data Processing

In [ ]:
#Overwriting our dataset in order to add context to mel frequencies
class UtteranceDatset_v2(torch.utils.data.Dataset):

  def __init__(self, x_file_path, y_file_path, context, train_mode = True):    
    self.context = context

    #Flatten and read data and pad 
    self.X_data = np.concatenate(np.load(x_file_path, allow_pickle=True), axis=0)

    if train_mode == True: 
      self.Y_data = np.concatenate(np.load(y_file_path, allow_pickle=True), axis=0).astype(np.int_)
      #Labels must match the size of training data 
      assert len(self.X_data) == len(self.Y_data),"Number of data != labels"
      self.Y_data = torch.from_numpy(self.Y_data)

    padding = np.zeros((context,40))
    self.X_data = np.concatenate((padding,self.X_data), axis = 0)
    self.X_data = np.concatenate((self.X_data, padding), axis = 0)
    self.X_data = self.X_data.astype(np.float32)
    self.X_data = torch.from_numpy(self.X_data)


  def __len__(self):
    return (len(self.X_data)-(2*self.context))
  
  def __getitem__(self,idx):
    if train_mode == True:
      #Index pattern used to account for padding 
      return (self.X_data[idx:2*self.context+idx+1, :].flatten(),self.Y_data[idx])
    else:
      return (self.X_data[idx:2*self.context+idx+1, :].flatten()) 


In [ ]:
class ModelComponents():
  def __init__(self):
    pass

  @staticmethod
  def create_data_loaders(train_data_path, train_lables_path, val_data_path, val_data_labels_path, test_data_path, context_size):
    
    #Creating Datasets
    training_data = UtteranceDatset_v2(train_data_path,
                                   train_lables_path,
                                   data_context_size)

    validation_data = UtteranceDatset_v2(val_data_path,
                                     val_data_labels_path,
                                     data_context_size)

    test_data = UtteranceDatset_Test_Data(test_data_path,
                                      data_context_size, train_mode = False)
    
    #Creating Dataloaders
    training_data_loader = torch.utils.data.DataLoader(training_data,
                                                   batch_size = 128,
                                                   shuffle = True,
                                                   num_workers = 1,
                                                   pin_memory = True)

    validation_data_loader = torch.utils.data.DataLoader(validation_data,
                                                   batch_size = 128,
                                                   shuffle = False,
                                                   num_workers = 1,
                                                   pin_memory = True)
    
    test_data_loader = torch.utils.data.DataLoader(test_data,
                                                   batch_size = 128,
                                                   shuffle = False,
                                                   num_workers = 1,
                                                   pin_memory = True)
    
    return(training_data_loader,validation_data_loader,test_data_loader)
    
data_context_size = 20 
training_data_loader, validation_data_loader, test_data_loader = ModelComponents.create_data_loaders("train.npy","train_labels.npy","dev.npy","dev_labels.npy","test.npy",data_context_size)

In [ ]:
#Refers to how many frames we pad our training example with
data_context_size = 20 

training_data = UtteranceDatset_v2("train.npy",
                                   "train_labels.npy",
                                   data_context_size)

validation_data = UtteranceDatset_v2("dev.npy",
                                     "dev_labels.npy",
                                     data_context_size)

test_data = UtteranceDatset_Test_Data("test.npy",
                                      data_context_size)


In [ ]:
training_data_loader = torch.utils.data.DataLoader(training_data,
                                                   batch_size = 128,
                                                   shuffle = True,
                                                   num_workers = 1,
                                                   pin_memory = True)

validation_data_loader = torch.utils.data.DataLoader(validation_data,
                                                   batch_size = 128,
                                                   shuffle = False,
                                                   num_workers = 1,
                                                   pin_memory = True)
test_data_loader = torch.utils.data.DataLoader(test_data,
                                                   batch_size = 128,
                                                   shuffle = False,
                                                   num_workers = 1,
                                                   pin_memory = True)

#Model Architecture

In [ ]:
class MLP(torch.nn.Module):

  def __init__(self, input_size, hidden_layer_size_0, hidden_layer_size_1, hidden_layer_size_2, hidden_layer_size_3, output_size):
    super(MLP,self).__init__()

    #Defining layers
    self.hidden_layer_0 = nn.Linear(input_size, hidden_layer_size_0)
    self.activation = nn.ReLU()
    self.bn_0 = nn.BatchNorm1d(num_features=hidden_layer_size_0)
    self.hidden_layer_1 = nn.Linear(hidden_layer_size_0, hidden_layer_size_1)
    self.bn_1 = nn.BatchNorm1d(num_features=hidden_layer_size_1)
    self.hidden_layer_2 = nn.Linear(hidden_layer_size_1, hidden_layer_size_2)
    self.bn_2 = nn.BatchNorm1d(num_features=hidden_layer_size_2)
    self.hidden_layer_3 = nn.Linear(hidden_layer_size_2, hidden_layer_size_3)
    self.bn_3 = nn.BatchNorm1d(num_features=hidden_layer_size_3)
    self.hidden_layer_4 = nn.Linear(hidden_layer_size_3,output_size)
    self.dropout = nn.Dropout(.5)
    
  def forward(self, data):
    #Hidden Layers
    output_hidden_layer_0 = self.hidden_layer_0(data)
    bn_0_output = self.bn_0(output_hidden_layer_0)
    output_activation_0 = self.activation(bn_0_output)
    output_drop_out_0 = self.dropout(output_activation_0)

    output_hidden_layer_1 = self.hidden_layer_1(output_drop_out_0)
    bn_1_output = self.bn_1(output_hidden_layer_1)
    output_activation_1 = self.activation(bn_1_output)
    output_drop_out_1 = self.dropout(output_activation_1)

    output_hidden_layer_2 = self.hidden_layer_2(output_drop_out_1)
    bn_2_output = self.bn_2(output_hidden_layer_2)
    output_activation_2 = self.activation(bn_2_output)
    output_drop_out_2 = self.dropout(output_activation_2)

    output_hidden_layer_3 = self.hidden_layer_3(output_drop_out_2)
    bn_3_output = self.bn_3(output_hidden_layer_3)
    output_activation_3 = self.activation(bn_3_output)
    output_drop_out_3 = self.dropout(output_activation_3)

    output_hidden_layer_4 = self.hidden_layer_4(output_drop_out_3)

    return(output_hidden_layer_4)

In [ ]:
#Model Parameters 
input_size = (1+2*data_context_size)*40
h_0_size = 2048
h_1_size = 2048
h_2_size = 1024
h_3_size = 512
output_size = 71 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

mlp_model = MLP(input_size, h_0_size, h_1_size,h_2_size,h_3_size,output_size)
mlp_model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr = .001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.9)

cuda:0


#Training 



In [ ]:
def SGD(total_epochs, model, loss_function, optimizer, train_loader, val_loader, scheduler):

  for epoch_num in range(total_epochs):
    train_loss_tracker = 0
    test_loss_tracker = 0
    starting_time = datetime.now()

    model.train()

    for x, y in train_loader:
      optimizer.zero_grad()

      x, y = x.to(device), y.to(device) 

      train_output = model(x)
      train_loss = loss_function(train_output, y)
      train_loss.backward()
      optimizer.step()

      train_loss_tracker = train_loss.item() + train_loss_tracker

    with torch.no_grad():
      model.eval()
      total_validation_examples = 0
      total_correct_validation_examples = 0

      for x, y in val_loader:
        x, y = x.to(device), y.to(device)

        #Calculating Loss
        val_output = model(x)
        val_loss = loss_function(val_output, y)
        test_loss_tracker = val_loss.item() + test_loss_tracker

        #Calculating Accuracy
        prediction_logits, prediction_classes = torch.max(val_output,1)
        total_validation_examples = total_validation_examples + x.shape[0]
        total_correct_validation_examples = total_correct_validation_examples + (prediction_classes==y).sum().item()
    
    scheduler.step()


    val_accuracy = (total_correct_validation_examples/total_validation_examples)

    train_loss_tracker = (train_loss_tracker/len(train_loader))
    test_loss_tracker = (test_loss_tracker/len(val_loader))

    epoch_runtime = datetime.now() - starting_time 
    print(f'Epoch: {epoch_num} Train Loss: {train_loss_tracker:.4f} Test Loss: {test_loss_tracker:.4f} Val Accuracy: {val_accuracy:.4f}')
    print(f'Epoch Runtime {epoch_runtime}')
    

In [ ]:
SGD(20, mlp_model, loss_function, optimizer, training_data_loader, validation_data_loader,scheduler)

Epoch: 0 Train Loss: 1.2368 Test Loss: 0.9462 Val Accuracy: 0.7089
Epoch Runtime 0:15:47.400045
Epoch: 1 Train Loss: 1.0565 Test Loss: 0.8883 Val Accuracy: 0.7261
Epoch Runtime 0:15:39.445480
Epoch: 2 Train Loss: 1.0038 Test Loss: 0.8542 Val Accuracy: 0.7351
Epoch Runtime 0:15:33.702358
Epoch: 3 Train Loss: 0.9726 Test Loss: 0.8353 Val Accuracy: 0.7413
Epoch Runtime 0:15:59.924188
Epoch: 4 Train Loss: 0.9507 Test Loss: 0.8198 Val Accuracy: 0.7460
Epoch Runtime 0:15:36.706162
Epoch: 5 Train Loss: 0.9341 Test Loss: 0.8064 Val Accuracy: 0.7501
Epoch Runtime 0:15:42.164236
Epoch: 6 Train Loss: 0.9212 Test Loss: 0.7979 Val Accuracy: 0.7519
Epoch Runtime 0:15:49.863296
Epoch: 7 Train Loss: 0.9107 Test Loss: 0.7910 Val Accuracy: 0.7541
Epoch Runtime 0:15:50.827251
Epoch: 8 Train Loss: 0.9013 Test Loss: 0.7819 Val Accuracy: 0.7567
Epoch Runtime 0:15:46.871681
Epoch: 9 Train Loss: 0.8939 Test Loss: 0.7834 Val Accuracy: 0.7571
Epoch Runtime 0:15:44.924151
Epoch: 10 Train Loss: 0.8873 Test Loss: 

#Saving Model Performance and Parameters


In [ ]:
def save_model(model, optimizer, scheduler_state_dict):
  torch.save({
  'model_state_dict': model.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'scheduler_state_dict' : scheduler.state_dict(),
  }, "/content/gdrive/My Drive/IDL/HW/HW1/P2/saves/model.pt")

In [ ]:
def predict_classes(model,test_loader):
  
  with torch.no_grad():
    model.eval()
    predicted_classes_total = []
    
    for x in test_loader:
      x = x.to(device)
      test_out = model(x)
      prediction_logits, prediction_classes = torch.max(test_out,1)
      prediction_classes = prediction_classes.tolist()
      predicted_classes_total.extend(prediction_classes)
    
    df = pd.DataFrame(predicted_classes_total)
    df.columns = ['label']
    path = '/content/gdrive/My Drive/IDL/HW/HW1/P2/saves/predicted_classes_yes.csv'
    df.to_csv(path)


In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
  
  checkpoint = torch.load(checkpoint_fpath)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  return model,optimizer


In [ ]:
model,optimizer = load_ckp("/content/gdrive/My Drive/IDL/HW/HW1/P2/saves/model.pt",mlp_model,optimizer)

In [ ]:
#save_model(mlp_model,optimizer,scheduler)
predict_classes(model,test_data_loader)